In [2]:
import pandas             as pd
import numpy              as np
import matplotlib.pyplot  as plt
from plyer import notification 
from datetime import datetime
import requests
import os
import zipfile

In [3]:
def baixar_e_extrair(url, nome_arquivo):
  
  # Verifica se o arquivo já existe
  if os.path.exists(nome_arquivo):
    print(f"O arquivo {nome_arquivo} já existe. Pulando o download.")
    return

  # Baixa o arquivo
  response = requests.get(url)
  response.raise_for_status()

  # Salva o arquivo no disco
  with open(nome_arquivo, 'wb') as f:
    f.write(response.content)

  # Extrai o arquivo ZIP
  with zipfile.ZipFile(nome_arquivo, 'r') as zip_ref:
    zip_ref.extractall()

# URL do arquivo ZIP:
url = "https://download.inep.gov.br/microdados/microdados_saeb_2021_ensino_fundamental_e_medio.zip"  
nome_arquivo = "microdados_saeb_2021_ensino_fundamental_e_medio.zip"  # Nome do arquivo baixado

# Chama a função para baixar e extrair o arquivo
baixar_e_extrair(url, nome_arquivo)

O arquivo microdados_saeb_2021_ensino_fundamental_e_medio.zip já existe. Pulando o download.


In [4]:
def alerta(nivel, base, etapa, erro=""):
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"

    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nível", nivel, "não disponível!")
        return

    # Enviar notificação
    notification.notify(
        title=title,
        message=msg,
        app_name='alerta',
        timeout=10
    )

In [5]:
def carregar_csv(caminho_csv):
    return pd.read_csv(caminho_csv, encoding='latin1', sep=';')

caminho_csv = "DADOS/TS_ESCOLA.csv"
df = carregar_csv(caminho_csv)
df.head(5)


,ID_SAEB,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_ESCOLA,IN_PUBLICA,ID_LOCALIZACAO,PC_FORMACAO_DOCENTE_INICIAL,PC_FORMACAO_DOCENTE_FINAL,...,MEDIA_5EF_LP,MEDIA_5EF_MT,MEDIA_9EF_LP,MEDIA_9EF_MT,MEDIA_EMT_LP,MEDIA_EMT_MT,MEDIA_EMI_LP,MEDIA_EMI_MT,MEDIA_EM_LP,MEDIA_EM_MT
0,2021,1,11,6316600,2,61324549,1,1,100.0,51.1,...,157.99,168.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,1,11,6316600,2,61335087,1,1,92.5,63.9,...,NaN,NaN,255.49,254.43,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,1,11,6316600,2,61336597,1,2,100.0,56.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,1,11,6316600,2,61342611,1,1,100.0,53.6,...,206.92,216.53,NaN,NaN,250.12,246.73,NaN,NaN,250.12,246.73
4,2021,1,11,6316600,2,61363380,1,1,87.5,26.9,...,170.73,169.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.info

<bound method DataFrame.info of        ID_SAEB  ID_REGIAO  ID_UF  ID_MUNICIPIO  ID_AREA  ID_ESCOLA  \
0         2021          1     11       6316600        2   61324549   
1         2021          1     11       6316600        2   61335087   
2         2021          1     11       6316600        2   61336597   
3         2021          1     11       6316600        2   61342611   
4         2021          1     11       6316600        2   61363380   
...        ...        ...    ...           ...      ...        ...   
70892     2021          5     53       6322169        1   61396990   
70893     2021          5     53       6322169        1   61397021   
70894     2021          5     53       6322169        1   61397022   
70895     2021          5     53       6322169        1   61397023   
70896     2021          5     53       6322169        1   61397024   

       IN_PUBLICA  ID_LOCALIZACAO  PC_FORMACAO_DOCENTE_INICIAL  \
0               1               1                        100.

In [7]:
df.shape

(70897, 137)

In [8]:
df.columns

Index(['ID_SAEB', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 'ID_AREA', 'ID_ESCOLA',
       'IN_PUBLICA', 'ID_LOCALIZACAO', 'PC_FORMACAO_DOCENTE_INICIAL',
       'PC_FORMACAO_DOCENTE_FINAL',
       ...
       'MEDIA_5EF_LP', 'MEDIA_5EF_MT', 'MEDIA_9EF_LP', 'MEDIA_9EF_MT',
       'MEDIA_EMT_LP', 'MEDIA_EMT_MT', 'MEDIA_EMI_LP', 'MEDIA_EMI_MT',
       'MEDIA_EM_LP', 'MEDIA_EM_MT'],
      dtype='object', length=137)

In [9]:
dados_ausentes = df.isna().sum()
dados_ausentes

ID_SAEB             0
ID_REGIAO           0
ID_UF               0
ID_MUNICIPIO        0
ID_AREA             0
                ...  
MEDIA_EMT_MT    64174
MEDIA_EMI_LP    70248
MEDIA_EMI_MT    70248
MEDIA_EM_LP     63736
MEDIA_EM_MT     63736
Length: 137, dtype: int64

In [10]:
def calcular_percentual_missing(df):
    # Calculando o percentual de valores ausentes
    percentual_missing = df.isna().sum() / len(df) * 100
    
    # Ordenando os resultados em ordem decrescente
    percentual_missing = percentual_missing.sort_values(ascending=False)
    
    return percentual_missing

In [11]:
calcular_percentual_missing(df)

NIVEL_3_MTEMI     99.084588
NIVEL_5_MTEMI     99.084588
NIVEL_1_LPEMI     99.084588
NIVEL_2_LPEMI     99.084588
NIVEL_10_MTEMI    99.084588
                    ...    
ID_REGIAO          0.000000
ID_ESCOLA          0.000000
IN_PUBLICA         0.000000
ID_LOCALIZACAO     0.000000
ID_SAEB            0.000000
Length: 137, dtype: float64

In [12]:
df.index

RangeIndex(start=0, stop=70897, step=1)

In [13]:
df.nunique().sort_values(ascending=True)

ID_SAEB               1
NIVEL_10_MTEM         1
IN_PUBLICA            1
NIVEL_10_MTEMT        1
NIVEL_9_MT9           1
                  ...  
MEDIA_9EF_LP       8079
MEDIA_9EF_MT       8437
MEDIA_5EF_MT      10045
MEDIA_5EF_LP      10107
ID_ESCOLA         70897
Length: 137, dtype: int64

In [14]:
type(df)

pandas.core.frame.DataFrame

In [15]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ID_SAEB,70897.0,2.021000e+03,0.000000,2021.00,2021.00,2021.00,2021.000,2021.00
ID_REGIAO,70897.0,2.706151e+00,1.069779,1.00,2.00,3.00,3.000,5.00
ID_UF,70897.0,3.079068e+01,9.837678,11.00,23.00,31.00,35.000,53.00
ID_MUNICIPIO,70897.0,6.319155e+06,1604.557983,6316600.00,6317669.00,6319157.00,6320429.000,6322169.00
ID_AREA,70897.0,1.873535e+00,0.332375,1.00,2.00,2.00,2.000,2.00
...,...,...,...,...,...,...,...,...
MEDIA_EMT_MT,6723.0,2.680258e+02,22.161414,185.81,253.57,267.09,281.275,399.52
MEDIA_EMI_LP,649.0,2.986970e+02,25.406534,203.98,282.06,300.99,317.720,349.23
MEDIA_EMI_MT,649.0,2.950209e+02,27.416555,213.74,275.12,295.38,313.860,370.11
MEDIA_EM_LP,7161.0,2.744996e+02,21.600875,190.66,260.62,273.71,287.380,363.71
